In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Define dataset path
dataset_path = '/content/drive/MyDrive/captured_landmarks2_combined'

# Labels for the classes
class_labels = {'left': 0, 'right': 1, 'up': 2, 'down': 3}

# Initialize arrays for data and labels
data = []
labels = []

# Load images and labels
for orientation in class_labels:
    orientation_path = os.path.join(dataset_path, orientation)
    for file_name in os.listdir(orientation_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(orientation_path, file_name)
            image = np.load(file_path)
            data.append(image)
            labels.append(class_labels[orientation])

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)


In [6]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Resize the images to MobileNet's expected input size (e.g., 224x224)
input_size = (224, 224)

def preprocess_images(images):
    images_resized = []
    for img in images:
        img_resized = tf.image.resize(img, input_size)
        img_resized = preprocess_input(img_resized)  # Normalize the input
        images_resized.append(img_resized)
    return np.array(images_resized)

X_train = preprocess_images(X_train)
X_val = preprocess_images(X_val)


In [7]:
from tensorflow.keras import layers, models

# Define the SSD MobileNet model
def build_ssd_mobilenet(input_shape):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

    # SSD specific layers
    ssd_output = layers.Conv2D(4, (1, 1), activation='softmax')(base_model.output)  # 4 orientations

    # Create the model
    model = models.Model(inputs=base_model.input, outputs=ssd_output)
    return model

# Build the model
input_shape = input_size + (3,)  # (224, 224, 3)
model = build_ssd_mobilenet(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
from tensorflow.keras import layers, models

# Define the MobileNetV2 model for classification
def build_mobilenet_model(input_shape):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

    # Add global average pooling to reduce the output to 1D
    x = layers.GlobalAveragePooling2D()(base_model.output)

    # Add a Dense layer for classification with 4 output classes (left, right, up, down)
    output = layers.Dense(4, activation='softmax')(x)  # 4 classes

    # Create the final model
    model = models.Model(inputs=base_model.input, outputs=output)
    return model

# Build the model
input_shape = input_size + (3,)  # (224, 224, 3)
model = build_mobilenet_model(input_shape)

# Compile the model for classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,  # You can adjust the number of epochs
    batch_size=32
)


Epoch 1/10
19/26 ━━━━━━━━━━━━━━━━━━━━ 43s 6s/step - accuracy: 0.5423 - loss: 1.0813

KeyboardInterrupt: 

In [11]:
def build_mobilenet_model(input_shape):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

    # Freeze the base model layers (only the top layers will be trained)
    base_model.trainable = False

    # Add global average pooling and a classification head
    x = layers.GlobalAveragePooling2D()(base_model.output)
    output = layers.Dense(4, activation='softmax')(x)  # 4 classes for left, right, up, down

    # Create the final model
    model = models.Model(inputs=base_model.input, outputs=output)
    return model

# Build the model with frozen base layers
input_shape = input_size + (3,)  # (224, 224, 3)
model = build_mobilenet_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [13]:
# Train the model for fewer epochs
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,  # Reduced number of epochs
    batch_size=32
)


Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.2223 - loss: 1.4045 - val_accuracy: 0.2233 - val_loss: 1.4220
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.2493 - loss: 1.4256 - val_accuracy: 0.2670 - val_loss: 1.3925
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2648 - loss: 1.4122 - val_accuracy: 0.2670 - val_loss: 1.3904
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.2522 - loss: 1.3964 - val_accuracy: 0.2670 - val_loss: 1.4204
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.2060 - loss: 1.4391 - val_accuracy: 0.2670 - val_loss: 1.3926
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.2435 - loss: 1.3995 - val_accuracy: 0.2233 - val_loss: 1.4074
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.2689 - loss: 1.3932 - val_accuracy: 0.2670 - val_loss: 1.3882
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.2508 - loss: 1.4042 - val_accuracy: 0.2670 - val_loss: